In [1]:
from qiskit import *
from qiskit.dagcircuit import DAGCircuit
from qiskit.tools.visualization import *
import networkx as nx

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import CXCancellation, CommutationTransformation, CommutationAnalysis, GateAnalysis, FixedPoint

ImportError: No module named 'marshmallow'

In [ ]:
q = QuantumRegister(7)
c = ClassicalRegister(2)
circ = QuantumCircuit(q, c)
circ.cx(q[0], q[1])
circ.cx(q[1], q[2])
circ.cx(q[2], q[3])
circ.cx(q[3], q[4])
circ.cx(q[4], q[5])
circ.cx(q[4], q[5])
circ.cx(q[3], q[4])
circ.cx(q[2], q[3])
circ.cx(q[1], q[2])
circ.cx(q[0], q[1])
circuit_drawer(circ)

In [ ]:
#DAG
qdag = DAGCircuit.fromQuantumCircuit(circ)
#print(qdag.basis)

#print(qdag.qregs)
#print(qdag.gates)
#print(qdag.num_tensor_factors())
#print(qdag.wire_type)
#print(qdag.collect_runs(["cx"]))
dag_drawer(qdag)

In [ ]:
# NX Multigraph

topo_order = list(nx.topological_sort(qdag.multi_graph)) 
for node in topo_order:
    if qdag.multi_graph.node[node]["type"] == "op":
        
        print(qdag.multi_graph.node[node]["qargs"], qdag.multi_graph.node[node].keys())
    print(node)
    print(qdag.multi_graph.node[node]["name"], qdag.multi_graph.node[node].keys())

print(qdag.multi_graph.node[22]['name'], qdag.multi_graph.node[22]['type'])
#print(CommutationAnalysis.commute(qdag.multi_graph.node[23], qdag.multi_graph.node[27]))
print(qdag.multi_graph.node[2].keys())

for pred in qdag.multi_graph.predecessors(21):
    print(pred)


In [ ]:
pm = PassManager()
pm.add_passes(GateAnalysis())
pm.add_passes(FixedPoint('gate_count'))
pm.add_passes([CXCancellation(), GateAnalysis(), FixedPoint('gate_count')],\
              do_while = lambda property_set: not property_set['fixed_point']['gate_count'])
# pm.add_passes(CommutationAnalysis())
# pm.add_passes(CommutationTransformation())

In [ ]:
from qiskit.transpiler import transpile
qdag = transpile(qdag, pass_manager=pm)
print(qdag.multi_graph.number_of_nodes())